# 数据采集网关（RTU）配置文件的数据结构设计

## 1.目的

梳理软件设计，便于程序理解和维护。

## 2.范围 

  适用于数据采集网关（RTU）及同类产品。  

## 3.背景

数据采集网关（RTU）的配置文件分为两部分。

南向（对下，终端设备）的配置文件描述子设备的拓扑关系，采集方式和子设备的测点。北向（对上，云平台）的配置文件描述云平台的接入方式和上报数据的格式。文件列表如下：

![jupyter](./picture/1.png)

其中，BUS_485_001和codes_template为南向配置文件，SERVER_001和SERVER_CODE为北向配置文件。

## 4.总体设计

配置文件的数据结构在整体上保持与配置文件内容的高度统一。采用面向对象的设计思想，通过逻辑分层把同类属性打包成基类（基本结构体）以提高程序的可读性。

此外，为了提高内存利用率，对于不会占用大块内存空间的数据（如从设备数据，一般小于10KB），采用数组的形式定义；对于会占用大块内存空间的数据（如点表，最多几百KB），采用链表的形式定义。

## 5.详细设计

### 5.1 南向配置文件的数据结构

#### 5.1.1 从设备数据结构

从设备数据结构定义如下：

In [ ]:
typedef struct _METER_PARAMETER_STRUCT
{
    METER_READ_ONLY_PARA_STRUCT     met_ro_para;
    METER_RW_PARA_STRUCT            met_rw_parameter;               // 预留
    SOUTH_USER_POINT_TABLE_t        *met_point_ptr;                 // 点表指针
}METER_PARAMETER_STRUCT;

其中，met_ro_para表示从设备的属性，met_point_ptr表示指向此从设备对应点表的指针。

METER_READ_ONLY_PARA_STRUCT结构体定义如下：

In [ ]:
typedef struct
{
    UINT16                  met_dev_code;                           // 设备号
    uart_parameters_t       uart_parameter;                         // 串口参数  
    UINT8                   met_protocol;                           // 从设备规约
    UINT32                  met_high_addr;                          // 从设备高地址
    UINT32                  met_low_addr;                           // 从设备低地址
    UINT8                   met_read_period;                        // 数据读取周期
    UINT8                   met_save_period;                        // 数据保存周期
    UINT16                  met_pointtable;                         // 从设备点表号
    advanced_parameters_t   advanced_parameter;                     // 从设备高级参数

    UINT8                   met_set_time_enable_flag : 1;           // 是否对时的标志
    UINT8                   met_FZ_state : 1;                       // 阀值事件的标志
    UINT8                   met_second_poll : 1;                    // 秒轮询开关
    UINT8                   met_run_enable_flag : 1;                // 设备是否投入运行标志
    UINT8                   met_single_write_flag : 1;              // modbus单点写入开关,0表示不使用05、06功能码
    UINT16                  option_length;                          // 可选内容长度（字节数）
    UINT8                   option_flag;                            // 可选内容标志
    INT8                    met_reserve_num[32];                    // 保留参数
    UINT8                   met_first_run_flag;                     // 表第一次运行
}METER_READ_ONLY_PARA_STRUCT;

#### 5.1.2 点表数据结构

点表数据结构采用了二维链表。如下图所示：

![jupyter](./picture/2.png)

其中第一维表示点表信息，包括点表号、点表版本号、点表测点个数等；第二维表示测点信息，包括测点数据类型、测点地址、寄存器个数等。

第一维点表信息结构体如下：

In [ ]:
typedef struct SOUTH_USER_POINT_TABLE_
{
    UINT32                          id;                             // 点表号
    INT8                            name[50];                       // 点表名 
    UINT16                          type;                           // 点表类型，保留
    INT8                            version[20];                    // 点表版本号
    UINT16                          measurepoint_number;            // 测点个数  
    SOUTH_USER_MEASURE_POINT_t      *p_measure_point;               // 指向首个测点
    struct SOUTH_USER_POINT_TABLE_  *p_next;                        // 指向下一个点表
}SOUTH_USER_POINT_TABLE_t;

第二维测点信息结构体如下：

In [ ]:
typedef struct SOUTH_USER_MEASURE_POINT_
{
    UINT16                              serial_number;              // 测点序号
    config_file_codes_template_code_t   measure_point_para;         // 测点数据
    struct SOUTH_USER_MEASURE_POINT_    *p_next;
}SOUTH_USER_MEASURE_POINT_t;

In [ ]:
typedef struct 
{
            // "code"       
            UINT8                   function_code;                  // 功能码
            UINT8                   read_mode;
            UINT16                  data_type;                      // 测点数据类型,char int等
            UINT16                  address_type;                   // 测点地址类型(主要用于部分plc)    
            UINT32                  address;                        // 测点地址
            UINT8                   length;                         // 寄存器个数
            UINT8                   offset;                         // 偏移量
            UINT8                   fast_upload_flag;               // 小循环标志位：1表示点位采取小循环，小轮训包括变化上送和数据告警
            float                   deadzone;                       // 死区
}config_file_codes_template_code_t;

### 5.2  北向配置文件的数据结构

#### 5.2.1 北向链路数据结构

北向链路数据结构定义如下：

In [ ]:
// 北向链路配置文件SERVER_001结构体
typedef struct
{
    // "App"
    app_t                   app;
    UINT16                  upload_period;
        // "Link"
        INT8                    trans_type[32+1];
        INT8                    trans_protocol[32+1];
        UINT16                  client_id;
        INT8                    username[32+1];
        INT8                    password[32+1];
            // "product_certificate"
            certificate_t           product_certificate;
        INT8                    url[128+1];
        UINT16                  port;
            // "more"
            INT8                    Ver[16+1];
            UINT8                   clear_session;
            INT8                    auth[64+1];
}config_file_server_001_t;

其中，product_certificate表示三元组信息。定义如下：

In [ ]:
// "product_certificate"基类
typedef struct
{
    INT8                    product_key[PRODUCT_KEY_LEN+1];
    INT8                    device_name[DEVICE_NAME_LEN+1];
    INT8                    device_secret[DEVICE_SECRET_LEN+1];
}certificate_t;

#### 5.2.2 北向业务数据结构

北向业务数据结构定义了业务相关的内容。不同的云平台对应的业务模型可能存在差异。此处以阿里云物模型为例。

北向业务数据结构一共二维。其中第一维是数组，表示从设备的业务数据；第二维是链表，表示对应从设备的测点业务数据。如下图所示：

![jupyter](./picture/3.png)

从设备业务数据结构如下：

In [ ]:
// 北向设备链表
typedef struct
{
    config_file_server_code_ied_t   device;                         // 从设备业务数据
    UINT16                          code_chain_num;                 // 测点个数
        // "code"
        north_slave_code_chain_t        *north_slave_code_chain;    // 指向首个测点业务数据
}north_slave_param_array_t;

In [ ]:
typedef struct
{
        // "IED"
        ied_t                           ied;                        // 对应文件"IED"层
            // "Para"
            config_file_server_code_param_t param;                  // ied的para
}config_file_server_code_ied_t;

In [ ]:
// 北向业务配置文件SERVER_CODE结构体
typedef struct
{
    UINT8                   device_order;                           // 该从设备的上传顺序号，预留
    UINT8                   protocol;                               // 协议索引号，用于方便下发控制，知道是什么协议
    UINT16                  period;                                 // 该从设备的上报周期，预留
    UINT8                   run_enable_flag;                        // 是否禁用
    INT8                    product_key[PRODUCT_KEY_LEN+1];         // 三元组
    INT8                    device_name[DEVICE_NAME_LEN+1];
    INT8                    device_secret[DEVICE_SECRET_LEN+1];
}config_file_server_code_param_t;

测点业务数据定义如下：

In [ ]:
typedef struct north_slave_code_chain_
{
    UINT16                          serial_number;                  // node序号
    config_file_server_code_code_t  parameter;                      // 参数
    float                           last_update_value;              // 上一次上报数据值
    struct north_slave_code_chain_  *p_next;
}north_slave_code_chain_t;

In [ ]:
typedef struct
{
            // "code"
            INT8                    identification[32+1];           // 测点标识符，上报唯一标识
            UINT8                   alarm_flag;                     // 数据是否告警
            float                   alarm_ceiling;                  // 告警上限
            float                   alarm_floor;                    // 告警下限
            UINT8                   burst_flag;                     // 数据是否变化上送
            float                   deadzone;                       // 死区
            INT8                    expression[32+1];               // 逻辑表达式
            UINT16                  upload_data_type;               // 上送数据类型,用于采集与上送类型不一致的情况,数据类型定义同南向
}config_file_server_code_code_t;

## 6.不足和改进方法

1.链表操作相比数组增加了一些复杂度。

2.业务数据结构设计以阿里云物模型为基础，在对接新平台时可能需要调整。